# 🎤 CosyVoice + vTTS API 테스트 (Kaggle)

**목표**: CosyVoice3 Zero-shot TTS를 vTTS API로 테스트

---

## 📌 사전 요구사항
- Kaggle GPU 활성화 필수! (Settings → Accelerator → GPU T4 x2)
- 예상 시간: 설치 ~15-20분, 테스트 ~5분

## 1️⃣ 환경 설정 및 설치 (~15-20분)

In [ ]:
# 📦 vTTS + CosyVoice 설치
import subprocess
import sys

print("📦 Installing vTTS with CosyVoice engine...\\n")

# numpy 호환성 확인
subprocess.run([sys.executable, "-m", "pip", "uninstall", "numpy", "-y", "-q"], capture_output=True)
subprocess.run([sys.executable, "-m", "pip", "install", "numpy>=1.24.0,<2.0.0", "-q"], capture_output=True)

# onnxruntime-gpu 설치 (Supertonic용 참조 오디오 생성)
subprocess.run([sys.executable, "-m", "pip", "uninstall", "onnxruntime", "onnxruntime-gpu", "-y", "-q"], capture_output=True)
subprocess.run([sys.executable, "-m", "pip", "install", "onnxruntime-gpu>=1.16.0", "-q"], capture_output=True)

# vTTS 설치 (Supertonic + CosyVoice)
result = subprocess.run(
    [sys.executable, "-m", "pip", "install", "-q",
     "vtts[supertonic-cosyvoice] @ git+https://github.com/bellkjtt/vTTS.git", "--no-deps"],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✅ vTTS 및 CosyVoice 의존성 설치 완료!")
    
    # 버전 확인
    import vtts
    print(f"vTTS version: {vtts.__version__ if hasattr(vtts, '__version__') else 'installed'}")
else:
    print(f"❌ 설치 실패:\\n{result.stderr}")

# CosyVoice 저장소 클론
print("\\n📥 Cloning CosyVoice repository...")
result = subprocess.run(
    ["git", "clone", "--depth", "1", "https://github.com/FunAudioLLM/CosyVoice.git", "/root/.vtts/CosyVoice"],
    capture_output=True,
    text=True
)

if result.returncode == 0 or "already exists" in result.stderr:
    print("✅ CosyVoice repository ready")
    
    # CosyVoice requirements 설치
    print("📦 Installing CosyVoice requirements...")
    subprocess.run(
        [sys.executable, "-m", "pip", "install", "-r", "/root/.vtts/CosyVoice/requirements.txt", "-q"],
        capture_output=True
    )
    print("✅ CosyVoice dependencies installed")
else:
    print(f"❌ Clone failed:\\n{result.stderr}")

# 환경변수 설정
import os
os.environ["COSYVOICE_PATH"] = "/root/.vtts/CosyVoice"

print("\\n✅ 설치 완료!")

## 2️⃣ 참조 오디오 생성 (Supertonic 사용)